In [2]:
!pip  install yfinance

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.8.tar.gz (948 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 770.7 kB/s eta 0:00:00m eta 0:00:010:00:01
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 719.7 kB/s eta 0:00:001m722.4 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 642.6 kB/s eta 0:00:000:00:010:00:0104
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
  Created wheel for

In [3]:
import yfinance as yf
import pandas as pd
import os

In [4]:
# List of Thai ESG stocks with '.BK' suffix for Yahoo Finance
stock_symbols = [
    'ACE', 'ADVANC', 'AH', 'AMATA', 'AOT', 'AP', 'ASK', 'AWC', 'BA', 'BAM',
    'BANPU', 'BBGI', 'BBL', 'BCH', 'BCPG', 'BDMS', 'BEM', 'BGRIM', 'BJC', 'BLA',
    'BTG', 'BTS', 'CBG', 'CENTEL', 'CK', 'CKP', 'COM7', 'CPALL', 'CPF', 'CPN',
    'CRC', 'DELTA', 'DITTO', 'DMT', 'EGCO', 'EPG', 'GLOBAL', 'GPSC', 'GULF',
    'GUNKUL', 'HANA', 'HENG', 'HMPRO', 'HTC', 'ICHI', 'III', 'ILM', 'INTUCH',
    'IVL', 'JMART', 'JMT', 'JTS', 'KBANK', 'KCE', 'KCG', 'KKP', 'KTB', 'KTC',
    'LH', 'MAJOR', 'MBK', 'MC', 'MEGA', 'MINT', 'MOSHI', 'MTC', 'NER', 'OR',
    'ORI', 'OSP', 'PLANB', 'PR9', 'PSH', 'PSL', 'PTT', 'PTTEP', 'PTTGC',
    'RATCH', 'RBF', 'RS', 'S', 'SABINA', 'SAK', 'SAPPE', 'SAT', 'SAWAD', 'SC',
    'SCB', 'SCC', 'SCCC', 'SCGD', 'SCGP', 'SGC', 'SHR', 'SIRI', 'SJWD', 'SMPC',
    'SNNP', 'SPALI', 'SSP', 'STA', 'STECON', 'STGT', 'SYNEX', 'TASCO', 'TCAP',
    'THANI', 'THCOM', 'TISCO', 'TLI', 'TOA', 'TOP', 'TQM', 'TTA', 'TTB', 'TTW',
    'TU', 'TVO', 'VGI', 'VIH', 'WHA', 'WHAUP'
]

# Append '.BK' for Yahoo Finance compatibility
stock_symbols = [symbol + '.BK' for symbol in stock_symbols]

# Define date range
start_date = "2022-01-01"
end_date = "2024-12-31"

# Create a directory to save individual stock data
output_dir = "Thai_ESG_Stock_Data"
os.makedirs(output_dir, exist_ok=True)

# Master file for combined stock data
summary_filename = os.path.join(output_dir, "Thai_ESG_Stock_Summary.csv")

# List to store stocks with no data
no_data_stocks = []

# DataFrame to store combined data
combined_data = pd.DataFrame()

# List to store summary statistics
summary_list = []

# Loop through each stock symbol and fetch data
for symbol in stock_symbols:
    try:
        print(f"Fetching data for {symbol}...")
        stock_data = yf.download(symbol, start=start_date, end=end_date)

        if not stock_data.empty:
            # Drop second and third row
            stock_data = stock_data.drop(stock_data.index[[1, 2]])

            # Change first column name to 'Date'
            stock_data.index.name = 'Date'

            # Save individual stock data to CSV
            filename = os.path.join(output_dir, f"{symbol}_2022_2024.csv")
            stock_data.to_csv(filename)
            print(f"✅ Data for {symbol} saved to {filename}")

            # Compute summary statistics
            mean_values = dict()
            mean_values["Stock"] = symbol
            num_rows = len(stock_data)
            mean_values["Num_Rows"] = num_rows
            mean_values['Close'] = round(float(stock_data['Close'].mean()), 2)
            mean_values['High'] = round(float(stock_data['High'].mean()), 2)
            mean_values['Low'] = round(float(stock_data['Low'].mean()), 2)
            mean_values['Open'] = round(float(stock_data['Open'].mean()), 2)
            mean_values['Volume'] = int(stock_data['Volume'].mean())

            # Append summary to list
            summary_list.append(mean_values)

        else:
            print(f"⚠️ No data found for {symbol}")
            no_data_stocks.append(symbol)

    except Exception as e:
        print(f"❌ Error fetching data for {symbol}: {e}")
        no_data_stocks.append(symbol)


# Convert summary statistics to DataFrame and save
if summary_list:
    summary_df = pd.DataFrame(summary_list)
    summary_df.to_csv(summary_filename, index=False)
    print(f"\n📊 Summary statistics saved to {summary_filename}")

# Print list of stocks with no data at the end
if no_data_stocks:
    print("\n⚠️ The following stocks had no data available:")
    print(no_data_stocks)
else:
    print("\n✅ All stocks had data available!")

print("✅ Data collection completed!")
display(summary_df)

Fetching data for ACE.BK...


[*********************100%***********************]  1 of 1 completed
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['Close'] = round(float(stock_data['Close'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['High'] = round(float(stock_data['High'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['Low'] = round(float(stock_data['Low'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipy

✅ Data for ACE.BK saved to Thai_ESG_Stock_Data/ACE.BK_2022_2024.csv
Fetching data for ADVANC.BK...
✅ Data for ADVANC.BK saved to Thai_ESG_Stock_Data/ADVANC.BK_2022_2024.csv
Fetching data for AH.BK...


[*********************100%***********************]  1 of 1 completed
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['Close'] = round(float(stock_data['Close'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['High'] = round(float(stock_data['High'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['Low'] = round(float(stock_data['Low'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipy

✅ Data for AH.BK saved to Thai_ESG_Stock_Data/AH.BK_2022_2024.csv
Fetching data for AMATA.BK...
✅ Data for AMATA.BK saved to Thai_ESG_Stock_Data/AMATA.BK_2022_2024.csv
Fetching data for AOT.BK...


...

✅ Data for WHA.BK saved to Thai_ESG_Stock_Data/WHA.BK_2022_2024.csv
Fetching data for WHAUP.BK...


[*********************100%***********************]  1 of 1 completed

✅ Data for WHAUP.BK saved to Thai_ESG_Stock_Data/WHAUP.BK_2022_2024.csv

📊 Summary statistics saved to Thai_ESG_Stock_Data/Thai_ESG_Stock_Summary.csv

✅ All stocks had data available!
✅ Data collection completed!



/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['Close'] = round(float(stock_data['Close'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['High'] = round(float(stock_data['High'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mean_values['Low'] = round(float(stock_data['Low'].mean()), 2)
/var/folders/cs/3lkp2pnd6j17xyfbtckgkhn40000gn/T/ipykernel_70060/1636383839.py:67: FutureWarning: Calling float on a sin

,Stock,Num_Rows,Close,High,Low,Open,Volume
0,ACE.BK,725,2.07,2.09,2.05,2.07,12531485
1,ADVANC.BK,725,207.70,209.27,206.04,207.68,5021037
2,AH.BK,725,24.32,24.72,23.96,24.36,1798702
3,AMATA.BK,725,21.61,21.87,21.37,21.62,6624567
4,AOT.BK,725,66.06,66.51,65.61,66.08,20221909
...,...,...,...,...,...,...,...
117,TVO.BK,725,23.27,23.45,23.10,23.28,858681
118,VGI.BK,725,3.26,3.32,3.20,3.26,52256061
119,VIH.BK,725,8.55,8.64,8.45,8.56,646849
120,WHA.BK,725,4.21,4.26,4.16,4.21,66189068
